# Generative Adversarial Network (GAN)

## Concept

GAN is composed of two neural networks that compete with each other, hence the name adversarial. There's the **Generator** network (G) and **Discriminator** network (D).

### Generator Network

The generator's task is to generate _fake data_ that could, hopefully, fool the discriminator. The generator does not have access to the _real data_ that the discriminator expects, instead it improves its ability to generate fake data by learning what feature of the data is expected by the discriminator.

- Input: vector of **noise**
- Output: fake data with the **same shape** as the real data

Stochastic gradient for generator network:

<center> $\bigtriangledown_{\theta_{d}}\frac{1}{m} \displaystyle\sum_{i=1}^{m}log (1-D(G(z^{(i)})))$ </center>

The target is to **minimize** the value of $1-D(G(z^{(i)}))$ which means **maximizing** the value of $D(G(z^{(i)})$, the probability that the discriminator passes the fake data generated by the generator as real.

### Discriminator Network

The discriminator's task is to determine if a data is _real_ or _fake_. The discriminator is actually updated by **ascending** its stochastic gradient, instead of descending.

- Input: real and fake **data**
- Output: binary (is the data real or fake)

Stochastic gradient for discriminator network:

<center> $\bigtriangledown_{\theta_{d}}\frac{1}{m} \displaystyle\sum_{i=1}^{m}[log D(x^{(i)}) + log (1-D(G(z^{(i)})))]$ </center>

## GAN for MNIST Dataset with Keras

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

class GAN():
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            
            # Plot the progress
            print ("%d [D loss: %f, acc_real: %.2f, acc_fake: %.2f] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                r, c = 3, 3

                fig, axs = plt.subplots(r, c)
                
                prediction = self.discriminator.predict(gen_imgs)
                prediction = np.round(prediction)
                
                failed_indices = []
                
                for i in range(len(prediction)):
                    if prediction[i] == 1:
                        failed_indices.append(i)
                
                failed_count = len(failed_indices)
                
                # Save fake images that the discriminator classified as real
                cnt = 0
                if (failed_count > 0):
                    for i in range(r):
                        for j in range(c):
                            # Prevent error where count exceeds the number of failed test case for discriminator
                            if (cnt >= failed_count):
                                break
                            axs[i,j].imshow(gen_imgs[failed_indices[cnt], :,:,0], cmap='gray')
                            axs[i,j].axis('off')
                            cnt += 1
                            
                fig.savefig("gan/images/mnist_%d_fooled.png" % epoch)
                plt.close()
                
                # Save fake images that the discriminator classified as fake
                fig, axs = plt.subplots(r, c)
                success_count = len(prediction) - failed_count
                cnt = 0
                if (success_count > 0):
                    for i in range(r):
                        for j in range(c):
                            # Find index not in faied_indices
                            while cnt in failed_indices:
                                cnt += 1
                                
                            # Prevent error where count exceeds the number of successful test case for discriminator
                            if (cnt >= success_count):
                                break
                            
                            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                            axs[i,j].axis('off')
                            cnt += 1
                fig.savefig("gan/images/mnist_%d_detected.png" % epoch)      
                plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=64, save_interval=250)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.856515, acc_real: 6.25, acc_fake: 100.00] [G loss: 0.914784]
1 [D loss: 0.350031, acc_real: 100.00, acc_fake: 84.38] [G loss: 0.970727]
2 [D loss: 0.292469, acc_real: 100.00, acc_fake: 71.88] [G loss: 1.053756]
3 [D loss: 0.239156, acc_real: 100.00, acc_fake: 93.75] [G loss: 1.155607]
4 [D loss: 0.194474, acc_real: 100.00, acc_fake: 90.62] [G loss: 1.296507]
5 [D loss: 0.170034, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.356003]
6 [D loss: 0.170873, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.480190]
7 [D loss: 0.148463, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.550474]
8 [D loss: 0.138297, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.634311]
9 [D loss: 0.122952, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.770716]
10 [D loss: 0.116916, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.856395]
11 [D loss: 0.106801, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.886899]
12 [D loss: 0.105350, acc_real: 100.00, acc_fake: 100.00] [G loss: 1.945119]
13 [D loss: 0.0

109 [D loss: 0.078394, acc_real: 100.00, acc_fake: 96.88] [G loss: 4.157787]
110 [D loss: 0.154310, acc_real: 96.88, acc_fake: 87.50] [G loss: 3.560243]
111 [D loss: 0.091832, acc_real: 100.00, acc_fake: 96.88] [G loss: 3.877174]
112 [D loss: 0.218451, acc_real: 90.62, acc_fake: 78.12] [G loss: 3.726089]
113 [D loss: 0.067223, acc_real: 100.00, acc_fake: 96.88] [G loss: 3.926586]
114 [D loss: 0.279747, acc_real: 93.75, acc_fake: 75.00] [G loss: 3.621866]
115 [D loss: 0.082212, acc_real: 100.00, acc_fake: 96.88] [G loss: 3.530175]
116 [D loss: 0.212466, acc_real: 100.00, acc_fake: 81.25] [G loss: 3.726063]
117 [D loss: 0.094509, acc_real: 100.00, acc_fake: 100.00] [G loss: 3.908247]
118 [D loss: 0.175724, acc_real: 100.00, acc_fake: 90.62] [G loss: 3.132327]
119 [D loss: 0.210435, acc_real: 100.00, acc_fake: 75.00] [G loss: 4.376843]
120 [D loss: 0.627728, acc_real: 81.25, acc_fake: 56.25] [G loss: 2.312982]
121 [D loss: 0.219641, acc_real: 100.00, acc_fake: 78.12] [G loss: 3.247863]
12

...

29510 [D loss: 0.650709, acc_real: 65.62, acc_fake: 62.50] [G loss: 0.794436]
29511 [D loss: 0.662248, acc_real: 59.38, acc_fake: 65.62] [G loss: 0.812150]
29512 [D loss: 0.715975, acc_real: 59.38, acc_fake: 50.00] [G loss: 0.806977]
29513 [D loss: 0.663582, acc_real: 46.88, acc_fake: 75.00] [G loss: 0.803146]
29514 [D loss: 0.693197, acc_real: 53.12, acc_fake: 50.00] [G loss: 0.840841]
29515 [D loss: 0.634538, acc_real: 71.88, acc_fake: 68.75] [G loss: 0.794758]
29516 [D loss: 0.681487, acc_real: 65.62, acc_fake: 53.12] [G loss: 0.835379]
29517 [D loss: 0.647074, acc_real: 59.38, acc_fake: 71.88] [G loss: 0.841748]
29518 [D loss: 0.684891, acc_real: 62.50, acc_fake: 43.75] [G loss: 0.857398]
29519 [D loss: 0.647985, acc_real: 53.12, acc_fake: 71.88] [G loss: 0.804520]
29520 [D loss: 0.668645, acc_real: 43.75, acc_fake: 59.38] [G loss: 0.844818]
29521 [D loss: 0.639965, acc_real: 68.75, acc_fake: 59.38] [G loss: 0.816820]
29522 [D loss: 0.633110, acc_real: 68.75, acc_fake: 53.12] [G lo

29617 [D loss: 0.679493, acc_real: 62.50, acc_fake: 53.12] [G loss: 0.815670]
29618 [D loss: 0.700907, acc_real: 53.12, acc_fake: 50.00] [G loss: 0.808003]
29619 [D loss: 0.678904, acc_real: 56.25, acc_fake: 50.00] [G loss: 0.837003]
29620 [D loss: 0.719605, acc_real: 50.00, acc_fake: 46.88] [G loss: 0.794398]
29621 [D loss: 0.647820, acc_real: 68.75, acc_fake: 53.12] [G loss: 0.908464]
29622 [D loss: 0.701524, acc_real: 53.12, acc_fake: 56.25] [G loss: 0.856863]
29623 [D loss: 0.699637, acc_real: 46.88, acc_fake: 59.38] [G loss: 0.768127]
29624 [D loss: 0.696541, acc_real: 50.00, acc_fake: 56.25] [G loss: 0.793852]
29625 [D loss: 0.676082, acc_real: 62.50, acc_fake: 56.25] [G loss: 0.793963]
29626 [D loss: 0.672868, acc_real: 50.00, acc_fake: 59.38] [G loss: 0.786929]
29627 [D loss: 0.652383, acc_real: 59.38, acc_fake: 62.50] [G loss: 0.811549]
29628 [D loss: 0.723755, acc_real: 56.25, acc_fake: 40.62] [G loss: 0.768906]
29629 [D loss: 0.711883, acc_real: 50.00, acc_fake: 53.12] [G lo

29723 [D loss: 0.659248, acc_real: 75.00, acc_fake: 53.12] [G loss: 0.806583]
29724 [D loss: 0.668616, acc_real: 62.50, acc_fake: 59.38] [G loss: 0.821886]
29725 [D loss: 0.730489, acc_real: 62.50, acc_fake: 46.88] [G loss: 0.799875]
29726 [D loss: 0.675124, acc_real: 59.38, acc_fake: 56.25] [G loss: 0.744434]
29727 [D loss: 0.657063, acc_real: 75.00, acc_fake: 50.00] [G loss: 0.761431]
29728 [D loss: 0.694956, acc_real: 56.25, acc_fake: 46.88] [G loss: 0.749931]
29729 [D loss: 0.704013, acc_real: 59.38, acc_fake: 40.62] [G loss: 0.802767]
29730 [D loss: 0.649591, acc_real: 68.75, acc_fake: 68.75] [G loss: 0.828378]
29731 [D loss: 0.655089, acc_real: 68.75, acc_fake: 59.38] [G loss: 0.829860]
29732 [D loss: 0.702473, acc_real: 50.00, acc_fake: 46.88] [G loss: 0.788788]
29733 [D loss: 0.675925, acc_real: 53.12, acc_fake: 56.25] [G loss: 0.850363]
29734 [D loss: 0.655285, acc_real: 68.75, acc_fake: 65.62] [G loss: 0.813717]
29735 [D loss: 0.678565, acc_real: 59.38, acc_fake: 56.25] [G lo

29830 [D loss: 0.698906, acc_real: 62.50, acc_fake: 43.75] [G loss: 0.770482]
29831 [D loss: 0.684224, acc_real: 56.25, acc_fake: 50.00] [G loss: 0.844879]
29832 [D loss: 0.680848, acc_real: 71.88, acc_fake: 53.12] [G loss: 0.814998]
29833 [D loss: 0.703710, acc_real: 43.75, acc_fake: 59.38] [G loss: 0.837487]
29834 [D loss: 0.716675, acc_real: 62.50, acc_fake: 46.88] [G loss: 0.792464]
29835 [D loss: 0.712283, acc_real: 50.00, acc_fake: 53.12] [G loss: 0.832041]
29836 [D loss: 0.720222, acc_real: 53.12, acc_fake: 50.00] [G loss: 0.805772]
29837 [D loss: 0.713331, acc_real: 46.88, acc_fake: 40.62] [G loss: 0.829569]
29838 [D loss: 0.706033, acc_real: 50.00, acc_fake: 56.25] [G loss: 0.834132]
29839 [D loss: 0.666645, acc_real: 56.25, acc_fake: 65.62] [G loss: 0.802077]
29840 [D loss: 0.649827, acc_real: 68.75, acc_fake: 56.25] [G loss: 0.835766]
29841 [D loss: 0.644111, acc_real: 59.38, acc_fake: 71.88] [G loss: 0.881277]
29842 [D loss: 0.691199, acc_real: 59.38, acc_fake: 56.25] [G lo

29937 [D loss: 0.645289, acc_real: 62.50, acc_fake: 71.88] [G loss: 0.788804]
29938 [D loss: 0.656371, acc_real: 59.38, acc_fake: 62.50] [G loss: 0.753892]
29939 [D loss: 0.662968, acc_real: 53.12, acc_fake: 62.50] [G loss: 0.802785]
29940 [D loss: 0.653011, acc_real: 62.50, acc_fake: 56.25] [G loss: 0.839308]
29941 [D loss: 0.726517, acc_real: 68.75, acc_fake: 37.50] [G loss: 0.826685]
29942 [D loss: 0.695672, acc_real: 37.50, acc_fake: 56.25] [G loss: 0.870501]
29943 [D loss: 0.710904, acc_real: 40.62, acc_fake: 53.12] [G loss: 0.818432]
29944 [D loss: 0.705260, acc_real: 40.62, acc_fake: 50.00] [G loss: 0.768622]
29945 [D loss: 0.715766, acc_real: 56.25, acc_fake: 43.75] [G loss: 0.801537]
29946 [D loss: 0.682503, acc_real: 65.62, acc_fake: 46.88] [G loss: 0.779031]
29947 [D loss: 0.660359, acc_real: 59.38, acc_fake: 59.38] [G loss: 0.821615]
29948 [D loss: 0.714787, acc_real: 43.75, acc_fake: 56.25] [G loss: 0.793872]
29949 [D loss: 0.660743, acc_real: 62.50, acc_fake: 59.38] [G lo